# Recommend Items Frequently Purchased Together
This notebook demonstrates how recommending items that are frequently purchased together is effective. The current best scoring public notebook [here][1] recommends to customers those customers' last purchases and scores public LB 0.020. In this notebook here, we will begin with that idea and add recommending items that are frequently purchased together with a customers' previous purchaes. This notebook improves the LB and scores LB 0.021. This notebook's strategy is as follows:
* recommend items previously purchased [idea here][1]
* recommend items that are bought together with previous purchases [idea here][2]
* recommend popular items [idea here][1]

[1]: https://www.kaggle.com/hengzheng/time-is-our-best-friend-v2
[2]: https://www.kaggle.com/cdeotte/customers-who-bought-this-frequently-buy-this

In [1]:
import cudf
import pandas as pd
import gc

print('RAPIDS version',cudf.__version__)

RAPIDS version 21.10.01


# Load Transactions, Reduce Memory
Discussion about reducing memory is [here][1]

[1]: https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations/discussion/308635

In [2]:
train = cudf.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')
train['customer_id'] = train['customer_id'].str[-16:].str.hex_to_int().astype('int64')
train['article_id'] = train.article_id.astype('int32')
train.t_dat = cudf.to_datetime(train.t_dat)
train = train[['t_dat','customer_id','article_id']]
train.to_parquet('train.pqt',index=False)
print( train.shape )
train.head()

(31788324, 3)


,t_dat,customer_id,article_id
0,2018-09-20,-6846340800584936,663713001
1,2018-09-20,-6846340800584936,541518023
2,2018-09-20,-8334631767138808638,505221004
3,2018-09-20,-8334631767138808638,685687003
4,2018-09-20,-8334631767138808638,685687004


In [3]:
tmp = train.groupby('customer_id').t_dat.max().reset_index()
tmp.columns = ['customer_id','max_dat']
train = train.merge(tmp,on=['customer_id'],how='left')
train['diff_dat'] = (train.max_dat - train.t_dat).dt.days
train = train.loc[train['diff_dat']<=6]
print('Train shape:',train.shape)

Train shape: (5181535, 5)


# Apriori Transformation:

### Jason Bian 2.28.2022
* https://towardsdatascience.com/the-frequently-bought-together-recommendation-system-b4ed076b24e5
* https://github.com/benbogart/frequently_bought_together/blob/main/Frequently_Bought_Together.ipynb

* Association_rules Notes: http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/

* CUDF cheatsheet: https://rapids.ai/assets/files/cheatsheet.pdf

* Other methods: clustering?

In [4]:
#train
train = train.loc[train.t_dat >= cudf.to_datetime('2020-09-16')]
train

,t_dat,customer_id,article_id,max_dat,diff_dat
31520544,2020-09-16,825696797860015377,874754016,2020-09-16,0
31520545,2020-09-16,825696797860015377,874754002,2020-09-16,0
31520546,2020-09-16,825696797860015377,935694002,2020-09-16,0
31520547,2020-09-16,-2424921637038555937,751567009,2020-09-16,0
31520548,2020-09-16,825696797860015377,717490075,2020-09-16,0
...,...,...,...,...,...
31788319,2020-09-22,5776383470964371557,573085057,2020-09-22,0
31788320,2020-09-22,8705515275876785235,890074001,2020-09-22,0
31788321,2020-09-22,-731468567251345846,868879003,2020-09-22,0
31788322,2020-09-22,8705515275876785235,886566001,2020-09-22,0


In [5]:
#Apriori treats duplicates as a single occurence
train_apri = train.groupby('customer_id')['article_id'].unique().reset_index()
train_apri

,customer_id,article_id
0,-9223100958908512198,"[673677024, 673677027, 785034009]"
1,-9223002534477110135,[793699001]
2,-9222810895170663723,"[892309001, 903306004]"
3,-9221924794303263774,"[708138013, 888343003, 892857002]"
4,-9221811157628158522,[891663002]
...,...,...
68979,9221607025486275188,"[699755071, 699755081, 751994003]"
68980,9221813808370389952,"[562245099, 865929002, 869331006]"
68981,9222310594107555341,"[884319001, 884319003]"
68982,9223099843213569889,[903924002]


In [6]:
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()

In [7]:
te.fit(train_apri['article_id'].to_pandas())
orders_1hot = te.transform(train_apri['article_id'].to_pandas())

orders_1hot = pd.DataFrame(orders_1hot, columns =te.columns_)
orders_1hot.head()

,108775044,111565001,111586001,111593001,111609001,123173001,129085001,129085027,130035001,144993001,...,948152002,949198001,949551001,949551002,949594001,952267001,952938001,953450001,953763001,956217002
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [8]:
from mlxtend.frequent_patterns import apriori

In [9]:
is_ap = apriori(orders_1hot, min_support=0.003, max_len=2, use_colnames=True)

In [10]:
is_ap.sort_values(by=['support'])

,support,itemsets
58,0.003001,(914441001)
7,0.003030,(573085043)
55,0.003030,(910601002)
3,0.003059,(456163060)
21,0.003059,(781613006)
...,...,...
34,0.006973,(866731001)
68,0.007437,(923758001)
70,0.007726,(924243002)
64,0.008248,(918522001)


# Calculate Association Rules

In [11]:
from mlxtend.frequent_patterns import association_rules

In [12]:
is_ap

,support,itemsets
0,0.003233,(372860001)
1,0.003131,(372860002)
2,0.005624,(448509014)
3,0.003059,(456163060)
4,0.003349,(456163086)
...,...,...
72,0.004088,(929165002)
73,0.004566,(929275001)
74,0.004247,(930380001)
75,0.004813,(934835001)


In [13]:
rules = association_rules(is_ap, metric="lift")
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


# Find Each Customer's Last Week of Purchases
Our final predictions will have the row order from of our dataframe. Each row of our dataframe will be a prediction. We will create the `predictionstring` later by `train.groupby('customer_id').article_id.sum()`. Since `article_id` is a string, when we groupby sum, it will concatenate all the customer predictions into a single string. It will also create the string in the order of the dataframe. So as we proceed in this notebook, we will order the dataframe how we want our predictions ordered.

# (1) Recommend Most Often Previously Purchased Items
Note that many operations in cuDF will shuffle the order of the dataframe rows. Therefore we need to sort afterward because we want the most often previously purchased items first. Because this will be the order of our predictons. Since we sort by `ct` and then `t_dat` will will recommend items that have been purchased more frequently first followed by items purchased more recently second.

In [14]:
tmp = train.groupby(['customer_id','article_id'])['t_dat'].agg('count').reset_index()
tmp.columns = ['customer_id','article_id','ct']
train = train.merge(tmp,on=['customer_id','article_id'],how='left')
train = train.sort_values(['ct','t_dat'],ascending=False)
train = train.drop_duplicates(['customer_id','article_id'])
train = train.sort_values(['ct','t_dat'],ascending=False)
train.head()

,t_dat,customer_id,article_id,max_dat,diff_dat,ct
38179,2020-09-16,-6341822073530655775,685814001,2020-09-16,0,30
80867,2020-09-18,954634701602309532,895555001,2020-09-18,0,26
169740,2020-09-21,-1032181102150868027,677930023,2020-09-21,0,25
77927,2020-09-18,-6439897037483747065,685813003,2020-09-19,1,25
106632,2020-09-18,6200564461742194660,715624001,2020-09-18,0,20


# (2) Recommend Items Purchased Together
In my notebook [here][1], we compute a dictionary of items frequently purchased together. We will load and use that dictionary below. Note that we use the command `drop_duplicates` so that we don't recommend an item that the user has already bought and we have already recommended above. We will need to use Pandas for some commands because RAPIDS cuDF doesn't have two conveinent commands, (1) create new column from dictionary map of another column (2) groupby aggregate strings sum.

We concatenate these rows after the rows containing customers' previous purchases. Therefore we will recommend previous items first and then items purchased together second. Note the trick to convert a column of int32 into a prediction string (using groupby agg str sum) is from notebook [here][2]

[1]: https://www.kaggle.com/cdeotte/customers-who-bought-this-frequently-buy-this
[2]: https://www.kaggle.com/hiroshisakiyama/recommending-items-recently-bought

In [15]:
# USE PANDAS TO MAP COLUMN WITH DICTIONARY
import pandas as pd, numpy as np
train = train.to_pandas()
pairs = np.load('../input/hmitempairs/pairs_cudf.npy',allow_pickle=True).item()
train['article_id2'] = train.article_id.map(pairs)

In [16]:
# RECOMMENDATION OF PAIRED ITEMS
train2 = train[['customer_id','article_id2']].copy()
train2 = train2.loc[train2.article_id2.notnull()]
train2 = train2.drop_duplicates(['customer_id','article_id2'])
train2 = train2.rename({'article_id2':'article_id'},axis=1)

In [17]:
# CONCATENATE PAIRED ITEM RECOMMENDATION AFTER PREVIOUS PURCHASED RECOMMENDATIONS
train = train[['customer_id','article_id']]
train = pd.concat([train,train2],axis=0,ignore_index=True)
train.article_id = train.article_id.astype('int32')
train = train.drop_duplicates(['customer_id','article_id'])

In [18]:
# CONVERT RECOMMENDATIONS INTO SINGLE STRING
train.article_id = ' 0' + train.article_id.astype('str')
preds = cudf.DataFrame( train.groupby('customer_id').article_id.sum().reset_index() )
preds.columns = ['customer_id','prediction']
preds.head()

,customer_id,prediction
0,-9223100958908512198,0673677024 0673677027 0785034009 0673677002 0...
1,-9223002534477110135,0793699001 0751542002
2,-9222810895170663723,0892309001 0903306004 0786187005
3,-9221924794303263774,0708138013 0888343003 0892857002 0741717002 0...
4,-9221811157628158522,0891663002 0706016001


# (3) Recommend Last Week's Most Popular Items
After recommending previous purchases and items purchased together we will then recommend the 12 most popular items. Therefore if our previous recommendations did not fill up a customer's 12 recommendations, then it will be filled by popular items.

In [19]:
train = cudf.read_parquet('train.pqt')
train.t_dat = cudf.to_datetime(train.t_dat)
train = train.loc[train.t_dat >= cudf.to_datetime('2020-09-16')]
top12 = ' 0' + ' 0'.join(train.article_id.value_counts().to_pandas().index.astype('str')[:12])
print("Last week's top 12 popular items:")
print( top12 )

Last week's top 12 popular items:
 0924243001 0924243002 0918522001 0923758001 0866731001 0909370001 0751471001 0915529003 0915529005 0448509014 0762846027 0714790020


# Write Submission CSV
We will merge our predictions onto `sample_submission.csv` and submit to Kaggle.

In [20]:
sub = cudf.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')
sub = sub[['customer_id']]
sub['customer_id_2'] = sub['customer_id'].str[-16:].str.hex_to_int().astype('int64')
sub = sub.merge(preds.rename({'customer_id':'customer_id_2'},axis=1),\
    on='customer_id_2', how='left').fillna('')
del sub['customer_id_2']
sub.prediction = sub.prediction + top12
sub.prediction = sub.prediction.str.strip()
sub.prediction = sub.prediction.str[:131]
sub.to_csv(f'submission.csv',index=False)
sub.head()

,customer_id,prediction
0,05be41580b991a22161251772826cc5af70fa38934c4a6...,0924243001 0924243002 0918522001 0923758001 08...
1,05bec9be7763c4f39aee019dd69dcf28ba0cacd194265b...,0924243001 0924243002 0918522001 0923758001 08...
2,05be821cc84fc5456e4b90d91e4084eab042c16c9bd5b7...,0924243001 0924243002 0918522001 0923758001 08...
3,05bf624b54514f043700e5d379f1921fd8866be5be0c35...,0924243001 0924243002 0918522001 0923758001 08...
4,05be47081cf99b5c4106d92ae4521030fb65e89ed7ad0a...,0924243001 0924243002 0918522001 0923758001 08...
